# Methods & interfaces

## structs & methods

In [1]:
import (
    "fmt"
    "math"
)

type Vertex struct {
    X, Y float64
}

func (v Vertex) Abs() float64 {
    return math.Sqrt(v.X*v.X + v.Y*v.Y)
}

v := Vertex{3, 4}
v.Abs()

5

In [2]:
type MyFloat float64

func (f MyFloat) Abs() float64 {
    if f < 0 {
        return float64(-f)
    }
    return float64(f)
}

f := MyFloat(-math.Sqrt2)
fmt.Println(f)
fmt.Println(f.Abs())
""

-1.4142135623730951
1.4142135623730951


In [3]:
type Vertex struct {
    X, Y float64
}

func (v Vertex) Abs() float64 {
	return math.Sqrt(v.X*v.X + v.Y*v.Y)
}

func (v *Vertex) Scale(f float64) { // use *Vertex pointer to change original value
	v.X = v.X * f
	v.Y = v.Y * f
}

v := Vertex{3, 4}
fmt.Println(v.Abs())
v.Scale(10)
fmt.Println(v.Abs())
""

5
50


In [4]:
func Abs(v Vertex) float64 {
	return math.Sqrt(v.X*v.X + v.Y*v.Y)
}

func Scale(v *Vertex, f float64) {
	v.X = v.X * f
	v.Y = v.Y * f
}


v := Vertex{3, 4}
fmt.Println(Abs(v))
Scale(&v, 10)
fmt.Println(Abs(v))
""

5
50


In [5]:
pntr := &v // p is pointer
*pntr      // * access pointer target

{30 40}

In [6]:
type Vertex struct {
	X, Y float64
}

func (v *Vertex) Scale(f float64) {
	v.X = v.X * f
	v.Y = v.Y * f
}

func (v *Vertex) Abs() float64 {
	return math.Sqrt(v.X*v.X + v.Y*v.Y)
}

v := &Vertex{3, 4}
fmt.Printf("Before scaling: %+v, Abs: %v\n", v, v.Abs())
v.Scale(5)
fmt.Printf("After scaling: %+v, Abs: %v\n", v, v.Abs())

nil

Before scaling: &{X:3 Y:4}, Abs: 5
After scaling: &{X:15 Y:20}, Abs: 25


## intrefaces

 An interface type is defined as a set of method signatures.

A value of interface type can hold any value that implements those methods.

Note: There is an error in the example code on line 22. Vertex (the value type) doesn't implement Abser because the Abs method is defined only on *Vertex (the pointer type). 

In [7]:
package main

import (
	"fmt"
	"math"
)

type Abser interface {
	Abs() float64
}

func main() {
	var a Abser
	f := MyFloat(-math.Sqrt2)
	v := Vertex{3, 4}

	a = f  // a MyFloat implements Abser
	a = &v // a *Vertex implements Abser

	// In the following line, v is a Vertex (not *Vertex)
	// and does NOT implement Abser.
	a = v

	fmt.Println(a.Abs())
}

type MyFloat float64

func (f MyFloat) Abs() float64 {
	if f < 0 {
		return float64(-f)
	}
	return float64(f)
}

type Vertex struct {
	X, Y float64
}

func (v *Vertex) Abs() float64 {
	return math.Sqrt(v.X*v.X + v.Y*v.Y)
}

main()

MissingMethod: comparing func() float64 against expected interface method func (interface{Abs() float64}).Abs() float64


ERROR: repl.go:22:2: error compiling assignment: a = v
	repl.go:22:2: incompatible types in assignment: main.Abser = main.Vertex
	reason: main.Vertex does not implement main.Abser: missing method func (interface{Abs() float64}).Abs() float64

In [8]:
type I interface {
	M()
}

type T struct {
	S string
}

// This method means type T implements the interface I,
// but we don't need to explicitly declare that it does so.
func (t T) M() {
	fmt.Println(t.S)
}

var i I = T{"hello"}
i.M()

hello


In [9]:
var i I
i = T{"test"}
i.M()

test



## Interface values

Under the hood, interface values can be thought of as a tuple of a value and a concrete type: 

`(value, type)`

 An interface value holds a value of a specific underlying concrete type.

Calling a method on an interface value executes the method of the same name on its underlying type. 

## Interface values with nil underlying values

If the concrete value inside the interface itself is nil, the method will be called with a nil receiver.

In some languages this would trigger a null pointer exception, but in Go it is common to write methods that gracefully handle being called with a nil receiver (as with the method M in this example.)

Note that an interface value that holds a nil concrete value is itself non-nil.


In [14]:
type I interface {
	M()
}

type T struct {
	S string
}

func (t *T) M() {
	if t == nil {
		fmt.Println("<nil>")
		return
	}
	fmt.Println(t.S)
}

func main() {
	var i I
	var t *T
	
	i = t // this t is nil
	describe(i) // prints <nil>
	i.M()

	i = &T{"hello"}
	describe(i) // prints hello
	i.M()
}

func describe(i I) {
	fmt.Printf("(%v, %T)\n", i, i)
}


 
## The empty interface

The interface type that specifies zero methods is known as the empty interface:

`interface{}`|

An empty interface may hold values of any type. (Every type implements at least zero methods.)

Empty interfaces are used by code that handles values of unknown type. For example, fmt.Print takes any number of arguments of type interface{}.

```go
	var i interface{}
	describe(i) // nil, nil

	i = 42
	describe(i) // 42, int

	i = "hello"
	describe(i) // hello, string
```

## Type switches

A type switch is a construct that permits several type assertions in series. 

```go
switch v := i.(type) {
case T:
    // here v has type T
case S:
    // here v has type S
default:
    // no match; here v has the same type as i
}
```

In [23]:
func do(i interface{}) {
	switch v := i.(type) {
	case int:
		fmt.Printf("Twice %v is %v\n", v, v*2)
	case string:
		fmt.Printf("%q is %v bytes long\n", v, len(v))
	default:
		fmt.Printf("I don't know about type %T!\n", v)
	}
}

do(21)
do("hello")
do(true)

Twice 21 is 42
"hello" is 5 bytes long
I don't know about type bool!


## Stringers


```go
type Stringer interface {
    String() string
}
```


In [29]:
type Person struct {
	Name string
	Age  int
}

func (p Person) String() string {
 return fmt.Sprintf("%v (%v years)", p.Name, p.Age)
}

a := Person{"Arthur Dent", 42}
z := Person{"Zaphod Beeblebrox", 9001}

fmt.Println(a)
""

{Arthur Dent 42}


In [ ]:
package main

import "fmt"

type IPAddr [4]byte

// TODO: Add a "String() string" method to IPAddr.

func main() {

}


In [121]:
package main

import (
	"fmt"
	"strings"
)

type IPAddr [4]byte

// TODO: Add a "String() string" method to IPAddr.
func (ip IPAddr) String() string {
	ipstr := make([]string, len(ip))
	for i, b := range ip {
		ipstr[i] = fmt.Sprintf("%d", b)
	}
	result := strings.Join(ipstr, ".")
	return result
}

func main() {
	hosts := map[string]IPAddr{
		"loopback":  {127, 0, 0, 1},
		"googleDNS": {8, 8, 8, 8},
	}
	for name, ip := range hosts {
		fmt.Printf("%v: %v\n", name, ip)
	}
}